In [1]:
%matplotlib inline


Sequence Models and Long-Short Term Memory Networks
===================================================

At this point, we have seen various feed-forward networks. That is,
there is no state maintained by the network at all. This might not be
the behavior we want. Sequence models are central to NLP: they are
models where there is some sort of dependence through time between your
inputs. The classical example of a sequence model is the Hidden Markov
Model for part-of-speech tagging. Another example is the conditional
random field.

A recurrent neural network is a network that maintains some kind of
state. For example, its output could be used as part of the next input,
so that information can propogate along as the network passes over the
sequence. In the case of an LSTM, for each element in the sequence,
there is a corresponding *hidden state* $h_t$, which in principle
can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model,
part-of-speech tags, and a myriad of other things.


## LSTM's in Pytorch

Before getting to the example, note a few things. Pytorch's LSTM expects
all of its inputs to be 3D tensors. The semantics of the axes of these
tensors is important. The first axis is the sequence itself, the second
indexes instances in the mini-batch, and the third indexes elements of
the input. We haven't discussed mini-batching, so lets just ignore that
and assume we will always have just 1 dimension on the second axis. If
we want to run the sequence model over the sentence "The cow jumped",
our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which
case the 1st axis will have size 1 also.

Let's see a quick example.



In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [11]:
import copy

lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
hidden_untouched = copy.deepcopy(hidden)

for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    
print("1. Step by step result")
print(out)
print(hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
inputs = torch.cat(inputs).view(len(inputs), 1, -1)  # Add the extra 2nd dimension
print("inputs.shape: ", inputs.shape)
hidden = hidden_untouched # reset the hidden state
out, hidden = lstm(inputs, hidden)
print("2. Entire sequence result. Should be equal")
print(out)
print(hidden)

1. Step by step result
tensor(1.00000e-02 *
       [[[ 1.2209, -6.0448,  7.1694]]])
(tensor(1.00000e-02 *
       [[[ 1.2209, -6.0448,  7.1694]]]), tensor([[[ 0.0780, -0.1016,  0.1758]]]))
inputs.shape:  torch.Size([5, 1, 3])
2. Entire sequence result. Should be equal
tensor([[[ 0.0491, -0.1457,  0.4626]],

        [[ 0.1360, -0.2683,  0.0916]],

        [[ 0.1420, -0.3027, -0.0995]],

        [[ 0.0361, -0.1282,  0.1582]],

        [[ 0.0122, -0.0604,  0.0717]]])
(tensor(1.00000e-02 *
       [[[ 1.2209, -6.0448,  7.1694]]]), tensor([[[ 0.0780, -0.1016,  0.1758]]]))


## Example: An LSTM for Part-of-Speech Tagging

In this section, we will use an LSTM to get part of speech tags. We will
not use Viterbi or Forward-Backward or anything like that, but as a
(challenging) exercise to the reader, think about how Viterbi could be
used after you have seen what is going on.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocab. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.


Prepare data:



In [39]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}
ix_to_tag = {0:"DET", 1:"NN", 2:"V"}
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

Create the model:



In [35]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, num_layer=1):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layer = num_layer
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        if num_layer > 1:
            self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layer, dropout=0.5)
        else:  # dropout only applicable for multi-layer LSTM
            self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layer)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(self.num_layer, 1, self.hidden_dim),
                torch.zeros(self.num_layer, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [45]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix), num_layer=1)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

total_loss = 0
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss
    if (epoch+1) % 100 == 0:
        print("average_loss: ", total_loss/100)
        total_loss = 0


# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)
    _, max_indexes = torch.max(tag_scores, 1)
    print(max_indexes)
    print([ix_to_tag[int(max_id)] for max_id in max_indexes])

tensor([[-1.2948, -0.7918, -1.2982],
        [-1.3133, -0.7489, -1.3541],
        [-1.4555, -0.6834, -1.3400],
        [-1.3798, -0.7148, -1.3507],
        [-1.3658, -0.7260, -1.3433]])
average_loss:  tensor(1.8979)
average_loss:  tensor(0.5980)
average_loss:  tensor(1.00000e-02 *
       9.4409)
tensor([[-0.0926, -2.7597, -3.6852],
        [-3.7546, -0.0259, -6.1221],
        [-4.0633, -4.1133, -0.0341],
        [-0.0167, -4.9343, -4.6739],
        [-4.5708, -0.0165, -5.1079]])
tensor([ 0,  1,  2,  0,  1])
['DET', 'NN', 'V', 'DET', 'NN']


## Exercise: Augmenting the LSTM part-of-speech tagger with character-level features


In the example above, each word had an embedding, which served as the
inputs to our sequence model. Let's augment the word embeddings with a
representation derived from the characters of the word. We expect that
this should help significantly, since character-level information like
affixes have a large bearing on part-of-speech. For example, words with
the affix *-ly* are almost always tagged as adverbs in English.

To do this, let $c_w$ be the character-level representation of
word $w$. Let $x_w$ be the word embedding as before. Then
the input to our sequence model is the concatenation of $x_w$ and
$c_w$. So if $x_w$ has dimension 5, and $c_w$
dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the
characters of a word, and let $c_w$ be the final hidden state of
this LSTM. Hints:

* There are going to be two LSTM's in your new model.
  The original one that outputs POS tag scores, and the new one that
  outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.
  The character embeddings will be the input to the character LSTM.




In [108]:
EMBEDDING_DIM = 5
HIDDEN_DIM = 6
CHAR_EMBEDDING_DIM = 3
CHAR_HIDDEN_DIM = 3

char_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        for char in word:
            if char not in char_to_ix:
                char_to_ix[char] = len(char_to_ix)

In [115]:
class CharLSTMTagger(nn.Module):

    def __init__(self, embedding_dim, char_hidden_dim, hidden_dim, vocab_size, tagset_size):
        super(CharLSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.char_hidden_dim = char_hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim+char_hidden_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence, char_encodings):
        embeds = self.word_embeddings(sentence)
        # concatenate the word embedding with character embedding
        embeds = torch.cat((embeds, char_encodings), 1) 
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [116]:
a = torch.randn(5, 4)
b = torch.randn(5, 3)
print(a)
print(b)
torch.cat((a,b), 1)

tensor([[ 3.0958,  0.5673, -0.0559,  0.1499],
        [-0.8122, -2.7369, -1.4221, -0.8618],
        [-0.8179,  1.6942, -1.1966, -0.9124],
        [-1.6377,  0.3533, -1.3037,  0.3014],
        [ 1.0951,  0.5102, -0.0772, -0.7530]])
tensor([[ 0.7179, -0.3594, -1.6003],
        [ 1.2272,  1.1749,  0.8943],
        [ 1.1426,  0.9445, -1.6470],
        [-1.0491,  0.6762, -0.6850],
        [-1.5369, -0.1945, -1.8869]])


tensor([[ 3.0958,  0.5673, -0.0559,  0.1499,  0.7179, -0.3594, -1.6003],
        [-0.8122, -2.7369, -1.4221, -0.8618,  1.2272,  1.1749,  0.8943],
        [-0.8179,  1.6942, -1.1966, -0.9124,  1.1426,  0.9445, -1.6470],
        [-1.6377,  0.3533, -1.3037,  0.3014, -1.0491,  0.6762, -0.6850],
        [ 1.0951,  0.5102, -0.0772, -0.7530, -1.5369, -0.1945, -1.8869]])

In [117]:
class CharEncoder(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(CharEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.char_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, word):
        embeds = self.char_embeddings(word)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(word), 1, -1), self.hidden)
        return lstm_out[-1].view(1,-1)

In [122]:
model = CharLSTMTagger(EMBEDDING_DIM,CHAR_HIDDEN_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
char_encoder = CharEncoder(CHAR_EMBEDDING_DIM, CHAR_HIDDEN_DIM, len(char_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
"""
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
"""

total_loss = 0
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        char_encoder.zero_grad()
        
        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)

        # Step 2.a encode char sequence
        char_encodings = []
        for word in sentence:
            char_encoder.hidden = char_encoder.init_hidden()
            char_in = prepare_sequence(word, char_to_ix)
            char_encoding = char_encoder(char_in)
            char_encodings.append(char_encoding)
        
        char_encodings = torch.cat(char_encodings)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in, char_encodings)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss
    if (epoch+1) % 100 == 0:
        print("average_loss: ", total_loss/100)
        total_loss = 0


# See what the scores are after training
with torch.no_grad():
    sentence = training_data[0][0]
    inputs = prepare_sequence(sentence, word_to_ix)
    char_encodings = []
    for word in sentence:
        char_encoder.hidden = char_encoder.init_hidden()
        char_in = prepare_sequence(word, char_to_ix)
        char_encoding = char_encoder(char_in)
        char_encodings.append(char_encoding)
        
    char_encodings = torch.cat(char_encodings)
    tag_scores = model(inputs, char_encodings)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)
    _, max_indexes = torch.max(tag_scores, 1)
    print(max_indexes)
    print([ix_to_tag[int(max_id)] for max_id in max_indexes])

average_loss:  tensor(1.9483)
average_loss:  tensor(0.7610)
average_loss:  tensor(0.1162)
tensor([[-0.0603, -2.9263, -5.3184],
        [-4.6133, -0.0257, -4.1692],
        [-3.0527, -3.7153, -0.0743],
        [-0.0246, -4.5323, -4.3012],
        [-5.3326, -0.0075, -5.9377]])
tensor([ 0,  1,  2,  0,  1])
['DET', 'NN', 'V', 'DET', 'NN']
